# Data Augmention using Generative LLMs


<center><img src="https://media.licdn.com/dms/image/v2/D4E12AQGLk5R0lcfr8A/article-cover_image-shrink_720_1280/article-cover_image-shrink_720_1280/0/1713883053110?e=1737590400&v=beta&t=0b0YHpIGPl8KxMzLfEhhwFbkcrT3ZLz2BDTaax-4HNM" alt="llama" style="width:60%"></a></center>

<br><br><br><br><br><br><br>
### Setting things up:

First, we need to set some things up. This usually starts by **creating a virtual environment**.

Open your preferred command line. Make sure **Python** is available. This means you should be able to enter the Python promp if you type ```Python```.

**Navigate/create a folder** where you want to run your system. 

Inside that folder you can run the following commands:

```
conda deactivate           # deactivate conda in case you have it active
python -m venv venv       # create a folder 'venv' that will contain your virtual environment
source venv/bin/activate   # activate your virtual environment; you should now see '(venv)' on the left of your terminal prompt
```


You are now inside your clean virtual enviroment: nothing except python should be available. 
**Let's start installing the necessary things**!

**Run the following commands** inside your terminal:

```
pip install llama-cpp-python
pip install openai
pip install sse_starlette
pip install starlette_context
pip install pydantic_settings
pip install "fastapi[all]"
pip install jupyter
```

We should everything we need to run LLMs locally...  We're almost there! 



#### What are we missing?


<br><br><br><br><br><br><br><br><br><br><br><br>
# An LLM!  
### (Where can we find them? Aren't they huge?)

<br><br><br><br><br><br><br><br><br><br><br><br>


# Quantization
#### What is it? 
[Read more here](https://medium.com/@techresearchspace/what-is-quantization-in-llm-01ba61968a51)




**Advantages:**
 - Lesser memory consumption: Lower bit width results in less memory for storage
 - Fast Inference: This is due to efficient computation due to its lower memory bandwidth requirements
 - Less energy consumption: Larger model need more data movement and storage resulting in more energy consumption. Hence a smaller model results in compartively lesser energy usage.
 - Smaller models: Can quantize to suit the need and deploy to device with samller hadware specifications.

**Disadvantages:**
 - Potential loss in accuracy due to squeezing of high precision weight to lower precision.

### We'll be using Quantized LLMs, in GGUF format
[Click here to know more about and how to find GGUF models](https://huggingface.co/docs/hub/en/gguf)

[QuantFactory](https://huggingface.co/QuantFactory) is a good place to start!

<br><br><br><br><br><br><br><br><br><br><br>

### Once you download your GGUF model, you should be able to run it (from within the virtual environment):

`
python -m llama_cpp.server --host 0.0.0.0 --model ./models/Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf --n_ctx 4096
`

This launches the **LLM as a server** that **can be used while it is running**.


# Using the LLM

In [6]:
from openai import OpenAI
from collections import defaultdict as dd

# Point to the server
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")


#client = OpenAI(base_url="http://130.37.53.128:9002/v1", api_key="cltl")


In [8]:
import openai

openai.api_request_timeout = 200  # Sets timeout to 120 seconds



## 1) In chat mode
- Try to understand how the contents of `history` is used
- What are the different `"roles"`?
- How to stop the chat?

In [12]:
history = [
    {"role": "system", "content": "You a professional translator. Provide translation. Try to be concise."},
]

while True:
    completion = client.chat.completions.create(
        model="local-model",
        messages=history,
        temperature=0.8,
        stream=True,
    )

    new_message = {"role": "assistant", "content": ""}

    for chunk in completion:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
            new_message["content"] += chunk.choices[0].delta.content

    history.append(new_message)
    print("\n")
    userinput = input("> ")
    if userinput.lower() in ["bye", "quit", "exit"]:
        print("\n BYE BYE!")
        break
    history.append({"role": "user", "content": userinput})
    
    print("\n")

APIConnectionError: Connection error.

## 2) In single prompt mode
- Is `history` still used?
- What does temperature do? What should theoretically happen if we set it to 0?


In [10]:
def query_LLM(model_client, prompt, temp=0.6):
    history = [
        {"role": "system", "content": prompt},
    ]
    completion = model_client.chat.completions.create(
        model="local-model", # this field is currently unused
        messages=history,
        temperature=temp,
        stream=False,
    )

    return completion.choices[0].message.content

In [40]:
# prompt = "What is prompt engineering?"
# print(query_LLM(client, prompt, temp=0.6))

In [39]:
prompt = "What is the best drink in the world? Your answer should contain the name of a single drink and nothing more."
print(query_LLM(client, prompt, temp=1))

APITimeoutError: Request timed out.

# Data Augmentation -- Machine Translation as a case study
1) We must know what we want
2) We must know if we can/how to ask it
3) We must learn how to process its output

## 1) Who can help me understand what you need for your assignment?

## 2) We must know if we can/how to ask it
- Am I using the right LLM for the task?
- Should we be using the chat or the single prompt method?

In [8]:
prompt = "Can you translate text to Russian: How are you?"
# prompt = "What languages were you trained with?"
print(query_LLM(client, prompt, temp=0.6))

NameError: name 'query_LLM' is not defined

In [12]:
prompt = "Translate the sentence from Russian into English language: Я ходил-ходил по лесу, но так ничего и не нашел."
# prompt = "What languages were you trained with?"
print(query_LLM(client, prompt, temp=0.6))

APITimeoutError: Request timed out.

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

## 3) We must learn how to process its output
- How do we stop all the yapping?
- What is the difference between zero-/one-/few-shot prompting?
- How do we get the ouput in the format we want?

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

In [47]:
# Working Example (Make sure you understand why we use a WHILE loop, and why we must use the TRY statement;) 
prompt = """
Please provide 5 english sentences and their respective portuguese translations. Each English sentence should be between 10 and 15 words long and must contain an idiom.
Your answer should be a list of list in python. The first element of each list should contain the English sentence and the second element should contain the Portuguese translation.
Provide only the list and nothing else. For example:
[["english sentence"], ["portuguese translation], ...]
"""

mt_list = []

import json

while len(mt_list) < 15:
    answer = query_LLM(client, prompt, temp=0.2)

    try:
        a = json.loads(answer)

        for item in a:
            mt_list.append(item)
    except:
        print("Failed to parse: ", answer)
        print("Trying again!\n\n\n")

print("\n\n\nDONE:")
for pair in mt_list:
    print(pair)
    print("\n")




DONE:
['The company is between a rock and a hard place with these financial decisions.', 'A empresa está entre uma pedra e um morro com essas decisões financeiras.']


["He's burning the midnight oil to meet this project deadline, but it's worth it.", 'Ele está queimando o óleo da meia-noite para cumprir a data-limite desse projeto, mas vale a pena.']


['The new policy is a double-edged sword for our business, both positive and negative impacts.', 'A nova política é uma espada de dois gumes para nossa empresa, tanto impactos positivos quanto negativos.']


["After the scandal, the company's reputation was left in tatters, a complete loss of trust.", 'Depois do escândalo, a reputação da empresa ficou em frangalhos, uma perda completa de confiança.']


['The team is feeling under the weather after that grueling project, they need some rest.', 'O time está se sentindo mal de saúde após aquele projeto agoniante, eles precisam de um pouco de descanso.']


['The company is between a rock

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

# Quality Estimation / "Evaluation" with BLEU

<center><img src="https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEhdPDCCJH2WVAuEWUvp-RWdQITk9L8dB2p62GVI9CzLHd_hC2cED4wovkTY07sSZmYHtiWcHbSUhPRzbg_2DYyHiq_9gElMN85ZwZAI2gPcuwQNleQATdqUlrd8klzjOLhvh-weaAWdqkA2/s1600/BLEU4.png" alt="llama" style="width:90%"></a><br><a href="https://kv-emptypages.blogspot.com/2019/04/understanding-mt-quality-bleu-scores.html">Taken from/Read more here</a></center>



<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

In [1]:
# !pip install nltk

# !pip install --upgrade nltk
# !pip uninstall nltk
!pip install nltk




In [7]:
import nltk
from nltk.util import ngrams
from nltk.translate.bleu_score import sentence_bleu
nltk.download('punkt_tab')
from inspect import signature

# # Replace the line where formatargspec is called
# signature = signature(model).parameters


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [106]:
#Original translation tokenization

import nltk
from nltk.util import ngrams

# Ensure that you have the required NLTK data
nltk.download('punkt')

# Read sentences from a file
with open('/Users/victoria/Desktop/blue/original_translation.txt', 'r', encoding='utf-8') as file:
    sentences = file.readlines()

# Process each sentence
for sentence in sentences:
    # Strip any leading/trailing whitespace and tokenize
    sentence = sentence.strip()
    if sentence:  # Ensure the sentence is not empty
        sentence_tokens = nltk.word_tokenize(sentence)
        ngrams_reference = []
        
        # Generate n-grams for n = 1, 2, 3, 4
        for n in [1, 2, 3, 4]:
            ngrams_reference += list(ngrams(sentence_tokens, n))
        
        # Print the n-grams for the current sentence
        print(f"N-grams for sentence '{sentence}':")
        print(ngrams_reference)


N-grams for sentence '"At the beginning of the 19th century , the Russian Empire extended from the Arctic Ocean in the north to the Black Sea on the south , from the Baltic Sea on the west to the Pacific Ocean , and ( until 1867 ) into Alaska in North America on the east ."':
[('``',), ('At',), ('the',), ('beginning',), ('of',), ('the',), ('19th',), ('century',), (',',), ('the',), ('Russian',), ('Empire',), ('extended',), ('from',), ('the',), ('Arctic',), ('Ocean',), ('in',), ('the',), ('north',), ('to',), ('the',), ('Black',), ('Sea',), ('on',), ('the',), ('south',), (',',), ('from',), ('the',), ('Baltic',), ('Sea',), ('on',), ('the',), ('west',), ('to',), ('the',), ('Pacific',), ('Ocean',), (',',), ('and',), ('(',), ('until',), ('1867',), (')',), ('into',), ('Alaska',), ('in',), ('North',), ('America',), ('on',), ('the',), ('east',), ('.',), ("''",), ('``', 'At'), ('At', 'the'), ('the', 'beginning'), ('beginning', 'of'), ('of', 'the'), ('the', '19th'), ('19th', 'century'), ('century'

[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [114]:
# source_sent = "le professeur est arrivé en retard à cause de la circulation"
# reference_transl = "the teacher arrived late because of the traffic"
# reference_transl_tok = nltk.word_tokenize(reference_transl)
# ngrams_reference = []
# for n in [1,2,3,4]:
#     ngrams_reference = ngrams_reference + list(ngrams(reference_transl_tok,n))

# print(ngrams_reference)

#C2 translation 
# import nltk
# from nltk.util import ngrams

# # Ensure that you have the required NLTK data
# nltk.download('punkt')

# # Read sentences from a file
# with open('/Users/victoria/Desktop/blue/C2_en_preprocessed_translation_2.txt', 'r', encoding='utf-8') as file:
#     sentences = file.readlines()

# # Process each sentence
# for sentence in sentences:
#     # Strip any leading/trailing whitespace and tokenize
#     sentence = sentence.strip()
#     if sentence:  # Ensure the sentence is not empty
#         sentence_tokens = nltk.word_tokenize(sentence)
#         ngrams_reference = []
        
#         # Generate n-grams for n = 1, 2, 3, 4
#         for n in [1, 2, 3, 4]:
#             ngrams_reference += list(ngrams(sentence_tokens, n))
        
#         # Print the n-grams for the current sentence
#         print(f"N-grams for sentence '{sentence}':")
#         print(ngrams_reference)





import nltk
from nltk.util import ngrams

nltk.download('punkt')

with open('/Users/victoria/Desktop/blue/C2_en_preprocessed_translation_2.txt', 'r', encoding='utf-8') as file:
    sentences = file.readlines()

all_ngrams = []

# Process each sentence
for sentence in sentences:
    # Strip any leading/trailing whitespace and tokenize
    sentence = sentence.strip()
    if sentence:  # Ensure the sentence is not empty
        sentence_tokens = nltk.word_tokenize(sentence)
        ngrams_sentence = []
        
        # Generate n-grams for n = 1, 2, 3, 4
        for n in [1, 2, 3, 4]:
            ngrams_sentence += list(ngrams(sentence_tokens, n))
        
        # Append the n-grams of the current sentence to the all_ngrams list
        all_ngrams.append(ngrams_sentence)

# n-grams for each sentence and store them for use as reference_transl_tok
for i, ngrams_sentence in enumerate(all_ngrams):
    print(f"N-grams for sentence {i + 1}:")
    print(ngrams_sentence)


reference_transl = all_ngrams[0]  # the first sentence's n-grams

print("\nN-grams for the selected reference sentence:")
print(reference_transl)


N-grams for sentence 1:
[('``',), ('The',), ('Russian',), ('Empire',), ('was',), ('the',), ('third',), ('largest',), ('of',), ('the',), ('ever',), ('existing',), ('states',), ('(',), ('after',), ('the',), ('British',), ('and',), ('Mongol',), ('empires',), (')',), ('extending',), ('to',), ('the',), ('North',), ('Ice',), ('Ocean',), ('in',), ('the',), ('north',), ('and',), ('the',), ('Black',), ('Sea',), ('in',), ('the',), ('south',), (',',), ('to',), ('the',), ('Baltic',), ('Sea',), ('in',), ('the',), ('west',), ('and',), ('the',), ('Pacific',), ('Ocean',), ('in',), ('the',), ('east',), ('.',), ("''",), ('``', 'The'), ('The', 'Russian'), ('Russian', 'Empire'), ('Empire', 'was'), ('was', 'the'), ('the', 'third'), ('third', 'largest'), ('largest', 'of'), ('of', 'the'), ('the', 'ever'), ('ever', 'existing'), ('existing', 'states'), ('states', '('), ('(', 'after'), ('after', 'the'), ('the', 'British'), ('British', 'and'), ('and', 'Mongol'), ('Mongol', 'empires'), ('empires', ')'), (')', 'ex

[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [110]:
# transl_list = [
#     "The professor was delayed due to the congestion",
#     "Congestion was responsible for the teacher being late",
#     "The teacher was late due to the traffic",
#     "The professor arrived late because circulation",
#     "The teacher arrived late because of the traffic"
# ]

# transl_list_tokenized = []
# for sent in transl_list:
#     transl_list_tokenized.append(nltk.word_tokenize(sent))

# print(transl_list_tokenized)



In [116]:
from nltk.translate.bleu_score import sentence_bleu

ngram_weights = (0.10, 0.30, 0.30, 0.30) # weights for 1-gram, 2-gram, 3-gram, 4-gram

for translation in transl_list_tokenized:

    # Fine the translation n-grams
    # Not needed for the score, just to see the overlap
    sent_ngrams = []
    for n in [1,2,3,4]:
        sent_ngrams = sent_ngrams + list(ngrams(translation,n))
    
    
    bleu_score1 = sentence_bleu([reference_transl], translation)  # This can be a list of references 
    bleu_score2 = sentence_bleu([reference_transl], translation, weights=ngram_weights) # This can be a list of references 

    print(bleu_score1, bleu_score2)

    #1-gram overlap
    print(set(ngrams_reference) & set(sent_ngrams))
    print("\n\n")

0 0
set()



0 0
{('for',)}



0 0
set()



0 0
set()



0 0
set()





In [112]:
# from nltk.translate.bleu_score import sentence_bleu

# ngram_weights = (0.10, 0.30, 0.30, 0.30) # weights for 1-gram, 2-gram, 3-gram, 4-gram

# for translation in transl_list_tokenized:

#     # Fine the translation n-grams
#     # Not needed for the score, just to see the overlap
#     sent_ngrams = []
#     for n in [1,2,3,4]:
#         sent_ngrams = sent_ngrams + list(ngrams(translation,n))
    
    
#     bleu_score1 = sentence_bleu([reference_transl_tok], translation)  # This can be a list of references 
#     bleu_score2 = sentence_bleu([reference_transl_tok], translation, weights=ngram_weights) # This can be a list of references 

#     print(bleu_score1, bleu_score2)

#     #1-gram overlap
#     print(set(ngrams_reference) & set(sent_ngrams))
#     print("\n\n")

1.0832677820940877e-231 1.052691193011681e-277
set()



7.176381577237209e-155 1.2950316234712509e-185
{('for',)}



7.711523862191631e-155 1.3328284280434942e-185
set()



4.1382219658909647e-78 1.695647221393335e-93
set()



0.8408964152537145 0.834236890454548
set()





### **BLUE score results**

In [118]:
import nltk
from nltk.util import ngrams
from nltk.translate.bleu_score import sentence_bleu

# Ensure that you have the required NLTK data
nltk.download('punkt')

# Read original translation sentences from a file
with open('/Users/victoria/Desktop/blue/original_translation.txt', 'r', encoding='utf-8') as file:
    original_sentences = file.readlines()

# Read preprocessed translation sentences from a file
with open('/Users/victoria/Desktop/blue/C2_en_preprocessed_translation_2.txt', 'r', encoding='utf-8') as file:
    preprocessed_sentences = file.readlines()

# List to store n-grams for each sentence in preprocessed translation
all_ngrams = []

# preprocessed sentence to extract n-grams
for sentence in preprocessed_sentences:
    sentence = sentence.strip()
    if sentence:
        sentence_tokens = nltk.word_tokenize(sentence)
        ngrams_sentence = []
        for n in [1, 2, 3, 4]:
            ngrams_sentence += list(ngrams(sentence_tokens, n))
        all_ngrams.append(ngrams_sentence)

# original sentence vs corresponding preprocessed sentence
for i, original_sentence in enumerate(original_sentences):
    original_sentence = original_sentence.strip()
    if original_sentence and i < len(preprocessed_sentences):
        preprocessed_sentence = preprocessed_sentences[i].strip()

        original_tokens = nltk.word_tokenize(original_sentence)
        preprocessed_tokens = nltk.word_tokenize(preprocessed_sentence)

        # n-grams for both sentences
        original_ngrams = []
        for n in [1, 2, 3, 4]:
            original_ngrams += list(ngrams(original_tokens, n))

        # the BLEU score for the current sentence pair
        reference = [original_tokens]  # BLEU expects a list of reference sentences
        bleu_score = sentence_bleu(reference, preprocessed_tokens)

        print(f"BLEU score for sentence {i + 1}: {bleu_score:.4f}")

        # overlap of n-grams between original and preprocessed sentences
        sent_ngrams = []
        for n in [1, 2, 3, 4]:
            sent_ngrams += list(ngrams(preprocessed_tokens, n))
        
        print("1-gram overlap:")
        print(set(original_ngrams) & set(sent_ngrams))
        
        print("\n2-gram overlap:")
        print(set(ngrams(original_tokens, 2)) & set(ngrams(preprocessed_tokens, 2)))
        
        print("\n3-gram overlap:")
        print(set(ngrams(original_tokens, 3)) & set(ngrams(preprocessed_tokens, 3)))
        
        print("\n4-gram overlap:")
        print(set(ngrams(original_tokens, 4)) & set(ngrams(preprocessed_tokens, 4)))
        
        print("\n\n")


[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


BLEU score for sentence 1: 0.1926
1-gram overlap:
{('Baltic',), (')',), ('the', 'Baltic', 'Sea'), ('Baltic', 'Sea'), ('the', 'east', '.'), ('the', 'Baltic'), ('North',), (',',), ('to', 'the'), ('Sea',), ('in', 'the', 'north'), ('east',), ('the', 'east'), ('the',), ('the', 'east', '.', "''"), ('.',), ("''",), ('in', 'the'), ('of',), ('``',), ('Empire',), ('to',), ('Black', 'Sea'), ('south', ','), ('north',), ('the', 'north'), ('the', 'Pacific', 'Ocean'), ('Black',), ('the', 'Black'), ('Pacific', 'Ocean'), ('Ocean', 'in', 'the', 'north'), ('Ocean', 'in'), ('south',), ('the', 'south'), ('Ocean', 'in', 'the'), ('Russian', 'Empire'), ('and',), ('Ocean',), ('.', "''"), ('Russian',), ('Pacific',), ('the', 'Pacific'), ('the', 'Black', 'Sea'), ('the', 'south', ','), ('(',), ('east', '.'), ('east', '.', "''"), ('west',), ('the', 'west'), ('of', 'the'), ('in',)}

2-gram overlap:
{('Black', 'Sea'), ('Baltic', 'Sea'), ('south', ','), ('the', 'Baltic'), ('.', "''"), ('in', 'the'), ('the', 'north'), 

In [120]:
import nltk
from nltk.util import ngrams
from nltk.translate.bleu_score import sentence_bleu

# Ensure that you have the required NLTK data
nltk.download('punkt')

# Read original translation sentences from a file
with open('/Users/victoria/Desktop/blue/original_translation.txt', 'r', encoding='utf-8') as file:
    original_sentences = file.readlines()

# Read preprocessed translation sentences from a file
with open('/Users/victoria/Desktop/blue/C2_en_preprocessed_translation_2.txt', 'r', encoding='utf-8') as file:
    preprocessed_sentences = file.readlines()

# Variables to hold cumulative statistics
total_bleu_score = 0
total_1gram_overlap = 0
total_2gram_overlap = 0
total_3gram_overlap = 0
total_4gram_overlap = 0
num_sentences = len(original_sentences)

# Iterate over sentences to compute statistics
for i, original_sentence in enumerate(original_sentences):
    original_sentence = original_sentence.strip()
    if original_sentence and i < len(preprocessed_sentences):
        preprocessed_sentence = preprocessed_sentences[i].strip()

        # Tokenize both sentences
        original_tokens = nltk.word_tokenize(original_sentence)
        preprocessed_tokens = nltk.word_tokenize(preprocessed_sentence)

        # Generate n-grams for both sentences
        original_ngrams = []
        preprocessed_ngrams = []
        for n in [1, 2, 3, 4]:
            original_ngrams += list(ngrams(original_tokens, n))
            preprocessed_ngrams += list(ngrams(preprocessed_tokens, n))

        # Calculate the BLEU score for the current sentence pair
        reference = [original_tokens]  # BLEU expects a list of reference sentences
        bleu_score = sentence_bleu(reference, preprocessed_tokens)
        total_bleu_score += bleu_score

        # Calculate n-gram overlap for 1-gram, 2-gram, 3-gram, and 4-gram
        total_1gram_overlap += len(set(ngrams(original_tokens, 1)) & set(ngrams(preprocessed_tokens, 1)))
        total_2gram_overlap += len(set(ngrams(original_tokens, 2)) & set(ngrams(preprocessed_tokens, 2)))
        total_3gram_overlap += len(set(ngrams(original_tokens, 3)) & set(ngrams(preprocessed_tokens, 3)))
        total_4gram_overlap += len(set(ngrams(original_tokens, 4)) & set(ngrams(preprocessed_tokens, 4)))

# Calculate average statistics
average_bleu_score = total_bleu_score / num_sentences
average_1gram_overlap = total_1gram_overlap / num_sentences
average_2gram_overlap = total_2gram_overlap / num_sentences
average_3gram_overlap = total_3gram_overlap / num_sentences
average_4gram_overlap = total_4gram_overlap / num_sentences

# Print out the statistics
print(f"Average BLEU score: {average_bleu_score:.4f}")
print(f"Average 1-gram overlap: {average_1gram_overlap:.2f}")
print(f"Average 2-gram overlap: {average_2gram_overlap:.2f}")
print(f"Average 3-gram overlap: {average_3gram_overlap:.2f}")
print(f"Average 4-gram overlap: {average_4gram_overlap:.2f}")


[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Average BLEU score: 0.1991
Average 1-gram overlap: 10.51
Average 2-gram overlap: 6.45
Average 3-gram overlap: 4.15
Average 4-gram overlap: 3.03
